In [3]:
import json
import pandas as pd
import numpy as np
import datetime
from datetime import datetime as dt
import altair as alt
import dash
from dash import dcc, html, Input, Output, ClientsideFunction
alt.data_transformers.enable("vegafusion")
app = dash.Dash(
    __name__,
    meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}],
    external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css']
)
app.title = "Spotify Song Popularity"

server = app.server
app.config.suppress_callback_exceptions = True


# Read data
df = pd.read_csv('../data/processed/spotify_songs_processed.csv', index_col=0)
object_columns = df.select_dtypes(include=['object']).columns
for column in object_columns:
    df[column] = df[column].astype('string')
features=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms']
genre_list = df['playlist_genre'].unique().tolist()
subgenre_list = df['playlist_subgenre'].unique().tolist()
artist_list = df['track_artist'].unique().tolist()

# Format release date
def parse_date(x):
    try:
        if len(x)==10:
            return dt.strptime(x, "%Y-%m-%d")
        elif len(x)==7:
            return dt.strptime(x, "%Y-%m")
        elif len(x)==4:
            return dt.strptime(x, "%Y")
    except ValueError:
        return None

df["track_album_release_date"] = df["track_album_release_date"].apply(parse_date)
# String -> Datetime

In [20]:
df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
Index: 32833 entries, 0 to 32832
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   track_id                  32833 non-null  string        
 1   track_name                32828 non-null  string        
 2   track_artist              32828 non-null  string        
 3   track_popularity          32833 non-null  int64         
 4   track_album_id            32833 non-null  string        
 5   track_album_name          32828 non-null  string        
 6   track_album_release_date  32833 non-null  datetime64[ns]
 7   playlist_name             32833 non-null  string        
 8   playlist_id               32833 non-null  string        
 9   playlist_genre            32833 non-null  string        
 10  playlist_subgenre         32833 non-null  string        
 11  danceability              32833 non-null  float64       
 12  energy                 

,track_popularity,track_album_release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,duration_min
count,32833.000000,32833,32833.000000,32833.000000,32833.000000,32833.000000,32833.000000,32833.000000,32833.000000,32833.000000,32833.000000,32833.000000,32833.000000,32833.000000,32833.000000
mean,42.477081,2011-08-07 17:30:40.185179648,0.654850,0.698619,5.374471,-6.719499,0.565711,0.107068,0.175334,0.084747,0.190176,0.510561,120.881132,225799.811622,3.763330
min,0.000000,1957-01-01 00:00:00,0.000000,0.000175,0.000000,-46.448000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4000.000000,0.066667
25%,24.000000,2008-08-25 00:00:00,0.563000,0.581000,2.000000,-8.171000,0.000000,0.041000,0.015100,0.000000,0.092700,0.331000,99.960000,187819.000000,3.130317
50%,45.000000,2016-08-26 00:00:00,0.672000,0.721000,6.000000,-6.166000,1.000000,0.062500,0.080400,0.000016,0.127000,0.512000,121.984000,216000.000000,3.600000
75%,62.000000,2019-04-19 00:00:00,0.761000,0.840000,9.000000,-4.645000,1.000000,0.132000,0.255000,0.004830,0.248000,0.693000,133.918000,253585.000000,4.226417
max,100.000000,2020-01-29 00:00:00,0.983000,1.000000,11.000000,1.275000,1.000000,0.918000,0.994000,0.994000,0.996000,0.991000,239.440000,517810.000000,8.630167
std,24.984074,NaN,0.145085,0.180910,3.611657,2.988436,0.495671,0.101314,0.219633,0.224230,0.154317,0.233146,26.903624,59834.006182,0.997233


In [11]:
import altair as alt

# Function for Danceability Chart
def create_danceability_chart(df):
    chart = alt.Chart(df).mark_bar().encode(
        alt.X("danceability:Q", bin=True),
        alt.Y('count()', stack=None),
        alt.Color('nominal_popularity:N')
    ).properties(
        title='Distribution of Danceability'
    )
    return chart.to_html()
#chart = alt.Chart(df).mark_bar().encode(
#        alt.X("danceability:Q", bin=True),
#        alt.Y('count()', stack=None),
#        alt.Color('nominal_popularity:N')
#    ).properties(
#        title='Distribution of Danceability'
#    )
#chart

alt.Chart(...)

In [17]:
# Function for Duration Chart
df['duration_min'] = df['duration_ms'] / 60000
def create_duration_chart(df):
    # Assuming 'duration_ms' is in milliseconds and we convert it to minutes
    chart = alt.Chart(df).mark_bar().encode(
        alt.X("duration_min:Q", bin=True),
        alt.Y('count()', stack=None),
        alt.Color('nominal_popularity:N')
    ).properties(
        title='Distribution of Duration (min)'
    )
    return chart

chart = alt.Chart(df).mark_bar().encode(
        alt.X("duration_min:Q", bin=True),
        alt.Y('count()', stack=None),
        alt.Color('nominal_popularity:N')
    ).properties(
        title='Distribution of Duration (min)'
    )
chart

alt.Chart(...)

In [22]:
# Function for Key Chart
def create_key_chart(df):
    chart = alt.Chart(df).mark_bar().encode(
        alt.X("key:O"),
        alt.Y('count()', stack=None),
        alt.Color('nominal_popularity:N')
    ).properties(
        title='Distribution of Key'
    )
    return chart
chart = alt.Chart(df).mark_bar().encode(
        alt.X("key:O"),
        alt.Y('count()', stack=None),
        alt.Color('nominal_popularity:N')
    ).properties(
        title='Distribution of Key'
    )
chart

alt.Chart(...)

In [23]:
# Function for Mode Chart
def create_mode_chart(df):
    chart = alt.Chart(df).mark_bar().encode(
        alt.X("mode:N"),
        alt.Y('count()', stack=None),
        alt.Color('nominal_popularity:N')
    ).properties(
        title='Distribution of Mode'
    )
    return chart
chart = alt.Chart(df).mark_bar().encode(
        alt.X("mode:N"),
        alt.Y('count()', stack=None),
        alt.Color('nominal_popularity:N')
    ).properties(
        title='Distribution of Mode'
    )
chart

alt.Chart(...)

In [26]:
chart = alt.Chart(df).mark_bar().encode(
    x='nominal_popularity',
    y='count()',
    color='nominal_popularity'
).properties(
    title='Population Level Distribution for All Songs'
)
chart

alt.Chart(...)